In [23]:
from pymongo import MongoClient
import pprint
MONGO_ATLAS_URI = open('.env','r').read()
client = MongoClient(MONGO_ATLAS_URI)
db = client['push']
goals = db['goals']
users = db['users']
goals_array = list(goals.find())
users_array = list(users.find())


In [27]:
def time_stamp_dict_create(time_stamp,record):
    tups = list(record.items()) + [('time_stamp',time_stamp)]
    return dict(tups)
time_stamps_array_grouped = [[time_stamp_dict_create(time_stamp,D) for time_stamp in D['timeStamps']] for D in goals_array]
time_stamp_array = [item for sublist in time_stamps_array_grouped for item in sublist]


In [46]:
from datetime import datetime,timedelta, date
from dateutil.parser import parse
import dateutil.parser
from dateutil.relativedelta import relativedelta

def calculate_month_range(dt):
    month = dt.strftime('%m')
    year = dt.strftime('%Y')
    beginning_of_month_text = "{}-{}-{}".format(str(year),str(month),'01')
    beginning_of_month = parse(beginning_of_month_text)
    next_month_beginning = beginning_of_month + relativedelta(months=1)
    end_of_month = next_month_beginning - timedelta(days=1)
    return [beginning_of_month,end_of_month]

def calculate_week_range(dt):
    start = dt - timedelta(days=dt.weekday()) - timedelta(days=1)
    end = start + timedelta(days=6)
    return [start , end]

def cadence_to_date_range(cadence):
    today = str(date.today())
    dt = parse(str(today))
    if cadence == 'Daily':
        return [dt,dt] 
    elif cadence == 'Weekly':
        return calculate_week_range(dt)
    elif cadence == 'Monthly':
        return calculate_month_range(dt)

    

[datetime.datetime(2020, 7, 5, 0, 0), datetime.datetime(2020, 7, 11, 0, 0)]


In [60]:
def determine_time_stamp_in_range(start,end,time_stamp):
    time_stamp = parse(time_stamp).strftime('%Y-%m-%d')
    start = start.strftime('%Y-%m-%d')
    end = end.strftime('%Y-%m-%d')
    return start <= time_stamp <= end

def determine_goal_overdue(goal_record):
    start,end = cadence_to_date_range(goal_record['cadence'])
    time_stamps = goal_record['timeStamps']
    filtered_time_stamps = [time_stamp for time_stamp in time_stamps if determine_time_stamp_in_range(start,end,time_stamp)]
    is_overdue = len(filtered_time_stamps) == 0
    return is_overdue
overdue_goals = [i for i in goals_array if determine_goal_overdue(i)]

77


In [2]:
import requests
payload = """{
    "to":"ExponentPushToken[6smFQ5K5I3kM6QNiJTZGAb]",
    "title":"title",
    "body":"abc",
    "sound": "default",
    "_displayInForeground": "true"
}"""

headers = {
  'Content-Type': 'application/json'
}
requests.post('https://exp.host/--/api/v2/push/send',data=payload,headers=headers)

<Response [200]>

In [14]:
users_with_expo_push_tokens = [user for user in users.find() if 'expoPushToken' in user.keys()]
users_with_expo_push_tokens_ids = [user['_id'] for user in users_with_expo_push_tokens]
goals_of_users_with_token = [goal for goal in goals.find() if goal['creatorID'] in users_with_expo_push_tokens_ids]

In [62]:
number_of_goals = len(goals_array)
number_of_users = len(users_array)
number_of_pushes = len(time_stamp_array)
number_of_goals_overdue =len(overdue_goals)
print(number_of_pushes)
print(number_of_users)
print(number_of_goals)
print(number_of_goals_overdue)


1160
30
109
77


In [15]:
for goal in goals.find():
    pprint.pprint(goal)

{'_id': ObjectId('5e9fbe235b2f8000240044a0'),
 'cadence': 'Daily',
 'cadenceCount': 1,
 'creationDate': '2020-04-22T03:46:43.699Z',
 'creatorID': ObjectId('5e9fb10585928b0024be726c'),
 'timeStamps': ['2020-04-21T23:58:05-04:00',
                '2020-04-22T15:11:05-04:00',
                '2020-04-23T14:54:17-04:00',
                '2020-04-25T21:16:01-04:00',
                '2020-04-25T21:16:02-04:00',
                '2020-04-25T21:16:02-04:00',
                '2020-04-25T21:16:03-04:00',
                '2020-04-29T22:01:17-04:00',
                '2020-04-30T09:50:21-04:00',
                '2020-05-03T21:27:47-04:00',
                '2020-05-03T21:27:50-04:00',
                '2020-05-03T21:27:53-04:00',
                '2020-05-03T21:28:58-04:00',
                '2020-05-03T21:29:00-04:00',
                '2020-05-03T21:29:02-04:00',
                '2020-05-03T21:29:04-04:00',
                '2020-05-17T21:18:35-04:00',
                '2020-05-18T17:46:49-04:00',
      

 'cadence': 'Daily',
 'cadenceCount': 1,
 'creationDate': '2020-06-12T06:27:01.045Z',
 'creatorID': ObjectId('5ee3202817286f0024e1ab5a'),
 'timeStamps': [],
 'title': 'Meditate'}
{'_id': ObjectId('5ee370c717286f0024e1ab5d'),
 'cadence': 'Daily',
 'cadenceCount': 1,
 'creationDate': '2020-06-12T12:10:47.353Z',
 'creatorID': ObjectId('5ee35e8b17286f0024e1ab5c'),
 'timeStamps': ['2020-06-12T13:06:31-04:00'],
 'title': 'Running'}
{'_id': ObjectId('5ebc33d9584257002440d1da'),
 'cadence': 'Daily',
 'cadenceCount': 1,
 'creationDate': '2020-05-13T17:52:25.887Z',
 'creatorID': ObjectId('5ea2324b56f2a60024df0978'),
 'timeStamps': ['2020-05-13T13:52:26-04:00',
                '2020-06-03T20:10:28-04:00',
                '2020-06-08T11:16:49-04:00',
                '2020-06-12T20:11:15-04:00',
                '2020-06-16T16:29:48-04:00',
                '2020-06-18T19:39:40-04:00',
                '2020-07-01T15:56:33-04:00',
                '2020-07-03T11:41:00-04:00'],
 'title': 'Workout '}
{'_

In [ ]:

# time_stamps = [item['timeStamps'] for sublist in goals.find() for item in sublist]
# number_of_pushes =len(list(time_stamps))

print(number_of_goals)